In [1]:
import pandas as pd
import ast
import re
import matplotlib.pyplot as plt
from collections import Counter
from collections import defaultdict

In [2]:
acquisition_type_list = [
    "EI",
    "MPI",
    ]

In [3]:
All_data_names = ['Reaction_CO_1.5h', 'Reaction_CO_7.5h', 'Reaction_CO_biphenyl', 'Reaction_CO_ortho', 'Reaction_CO_Cl', 'Reaction_CS', 
                  'Reaction_CN', 'Reaction_2+2', 'Reaction_CF3', 'Reaction_CH2CF3', 'Reaction_CH2F', 'Reaction_Cy', 'Reaction_OCH2F', 'Reaction_SCF3',
                 'Reaction_Masuda', 'Reaction_Sumida']

threshold_dict = {'Reaction_CO_1.5h': 80,
                  'Reaction_CO_7.5h': 90,
                  'Reaction_CO_biphenyl': 70,
                  'Reaction_CO_ortho': 50,
                  'Reaction_CO_Cl': 60,
                  'Reaction_CS': 90, 
                  'Reaction_CN': 90,
                  'Reaction_2+2': 70,
                  'Reaction_CF3': 90,
                  'Reaction_CH2CF3': 70,
                  'Reaction_CH2F': 50,
                  'Reaction_Cy': 90,
                  'Reaction_OCH2F': 70,
                  'Reaction_SCF3': 90,
                  'Reaction_Masuda': 70,
                  'Reaction_Sumida': 65
                 }

In [4]:
# 元データを保持する辞書
original_data_dict = {}
for target in All_data_names:
    original_data_dict[f'{target}'] = pd.read_csv(f'../../../data/data_60_{target}.csv')

for target in All_data_names:
    print("~*~" * 15)
    print(f"target: {target}")

    for acquisition_type in acquisition_type_list:    
        #結果読み込み
        file_name = f'BayesianOptimization_{target}_GPyOpt_{acquisition_type}_RBF_result'
        df = pd.read_csv(f'../../results/{file_name}.csv')
        
        # str型からint型に変換し、選択した順番のindices_listのdfを作成
        converted_df = pd.DataFrame()
        converted_selected_list = []
        converted_initial_list = []
    
        for selected_str in df['selected_indices_per_step']:
            # 文字列をリストに変換し、各要素を整数に変換
            selected_list = ast.literal_eval(selected_str)
            selected_list = list(map(int, selected_list))
            # 変換後のリストを追加
            converted_selected_list.append(selected_list)
            
        converted_df['selected_indices_per_step'] = converted_selected_list
    
        for initial_str in df['initial_indices']:
            # 文字列をリストに変換し、各要素を整数に変換
            initial_list = ast.literal_eval(initial_str)
            initial_list = list(map(int, initial_list))
            # 変換後のリストを追加
            converted_initial_list.append(initial_list)
            
        converted_df['initial_indices'] = converted_initial_list 
    
        converted_df['indices_list'] = converted_df['initial_indices'] + converted_df['selected_indices_per_step']
    
        #indices_listをもとに選択した触媒を用いたときの収率lsitを作成
    
        # 各行のリストに従い、参照データフレームから対応する値を取得して新しい列を作成
        converted_df["selected_Yield"] = converted_df["indices_list"].apply(
            lambda indices: [original_data_dict[f'{target}'].loc[idx, "Yield"] for idx in indices]
        )
    
        first_select = []
        last_select = []
        
        selected_df = pd.DataFrame()
        selected_list = []
        
        for actual_rewards_per_step in converted_df['selected_Yield']:
            
            indices_plus_one = [i + 1 for i, num in enumerate(actual_rewards_per_step) if num >= threshold_dict[f'{target}']]
            print(f'収率{threshold_dict[f'{target}']}％以上を選択したstepのリスト: {indices_plus_one}')
            selected_list.append(indices_plus_one)
        
        selected_df['selected_list'] = selected_list
        
        #しきい値以上の収率を持つ触媒を選んだステップ数の保存   
        select_averages = [sum(values) / len(values) for values in zip(*selected_df['selected_list'])]
        
        i = 1
        for select_average in select_averages:
            print(f'{i}回目の収率{threshold_dict[f'{target}']}％以上の触媒を選択したstep平均: {select_average}')
            i += 1
        
        df_select_averages = pd.DataFrame()
        df_select_averages[f'{file_name}'] = select_averages
        df_select_averages.to_csv(f'../results/{file_name}_select_averages.csv', index=False)

~*~~*~~*~~*~~*~~*~~*~~*~~*~~*~~*~~*~~*~~*~~*~
target: Reaction_CO_1.5h
収率80％以上を選択したstepのリスト: [10, 12, 13, 15, 31, 43, 55, 60]
収率80％以上を選択したstepのリスト: [1, 9, 10, 13, 19, 25, 27, 59]
収率80％以上を選択したstepのリスト: [11, 12, 13, 17, 28, 46, 53, 59]
収率80％以上を選択したstepのリスト: [10, 15, 16, 17, 19, 32, 49, 59]
収率80％以上を選択したstepのリスト: [10, 12, 13, 17, 26, 32, 43, 60]
収率80％以上を選択したstepのリスト: [12, 14, 15, 19, 26, 29, 59, 60]
収率80％以上を選択したstepのリスト: [1, 13, 18, 19, 23, 27, 28, 59]
収率80％以上を選択したstepのリスト: [13, 20, 23, 24, 25, 31, 48, 60]
収率80％以上を選択したstepのリスト: [13, 21, 27, 33, 34, 35, 59, 60]
収率80％以上を選択したstepのリスト: [1, 13, 20, 21, 38, 39, 51, 59]
収率80％以上を選択したstepのリスト: [1, 17, 19, 23, 26, 54, 59, 60]
収率80％以上を選択したstepのリスト: [1, 16, 27, 30, 36, 47, 54, 59]
収率80％以上を選択したstepのリスト: [8, 13, 26, 27, 28, 34, 47, 60]
収率80％以上を選択したstepのリスト: [6, 11, 22, 23, 24, 29, 53, 60]
収率80％以上を選択したstepのリスト: [1, 16, 19, 21, 22, 26, 30, 41]
収率80％以上を選択したstepのリスト: [10, 16, 17, 18, 28, 31, 42, 60]
収率80％以上を選択したstepのリスト: [1, 12, 16, 18, 24, 29, 33, 45]
収率80